In [43]:
import os
import re
import numpy as np
import torch
import torch.nn as nn
from torch.utils import data
from glob import glob

# Prepare Dataset

In [29]:
POSE_FEATURE_PATH = "D:/CU Files/IoT/Featurized_dataset/"
IMAGE_FEATURE_PATH = "D:/CU Files/IoT/image_feature/resnet50/"

def get_path_dict(pose_feature_path,
                  image_feature_path):
    all_pose_path = [y for y in glob(os.path.join(pose_feature_path, "*.mp4.npz"))]
    all_pose_name = [re.findall("Featurized_dataset\\\\(.+).mp4.npz", path)[0]
                     for path in all_pose_path]
    all_image_path = [y for y in glob(os.path.join(image_feature_path, "*.mp4.npz"))]
    all_image_name = [re.findall(image_feature_path[:-1] + "\\\\(.+).mp4.npz", path)[0] for path in all_image_path]
    name_intersection = list(set(all_pose_name).intersection(set(all_image_name)))

    print("{} pose feature files".format(len(all_pose_name)))
    print("{} image feature files".format(len(all_image_name)))
    print("{} feature files available".format(len(name_intersection)))


    pi_path_dict = {}   # get a dictionary which records the pose and image feature path
    for i, feature_name in enumerate(all_pose_name):
        try:
            idx = all_image_name.index(feature_name)
        except:
            continue
        pose_path = all_pose_path[i]
        image_path = all_image_path[idx]
        pi_path_dict[feature_name] = (pose_path, image_path)

    return pi_path_dict

In [31]:
path_dict = get_path_dict(pose_feature_path=POSE_FEATURE_PATH, image_feature_path=IMAGE_FEATURE_PATH)

977 pose feature files
969 image feature files
969 feature files available


In [45]:
for feature_name, (pose_path, image_path) in path_dict.items():
    pose_file = np.load(pose_path, allow_pickle=True)
    print(pose_file.files)
    break

['boxes', 'segments', 'keypoints', 'metadata']


In [35]:
class mydataset(data.Dataset):
    def __init__(self, path_dict):
        self.path_dict = path_dict
        self.Data, self.Label = self._get_features()

    def _get_features(self, pose=True, image=True):
        features = []
        labels = []
        self.__label_encoder = {'no_interaction':0,
                                'open_close_fridge':1,
                                'put_back_item':2,
                                'screen_interaction':3,
                                'take_out_item':4}
        for feature_name, (pose_path, image_path) in path_dict.items():
            label = '_'.join(feature_name.split('_')[:-3])
            labels.append(label)
            this_feature = self._get_single_feature(pose_path, image_path, pose, image)

    def _get_single_feature(self, pose_path, image_path, pose, image):
        pass


    def __getitem__(self, index):
        x = self.X[index]
        label = self.Label[index]
        return x, label

    def __len__(self):
        return len(self.Label)

In [36]:
train_dataset = mydataset()
train_loader = data.DataLoader(train_dataset, batch_size=100, shuffle=True)

data is read


In [37]:
for epoch in range(5):
    for _ in train_loader:
        continue